Analyser le graphe de connaissance est difficile car il n'y a pas de métrique idéale. La véracité des informations ? Leur liens ? Leur cohérance ? Toutes les métriques imaginables relèvent plus de la notion d'expert et n'est pas donc pas implémentable à la volée de manière calculatoire. Il existe cependant des métriques non supervisées qui permettent de comparer l'évolution d'une graphe. Entre 2 itérations si la valeur de la métrique change on peut savoir si c'est mieux ou moins bien ! Ces métriques n'ont pas été implémentées mais il serait interessant de s'y pencher.

# Evaluation Metrics
We currently have no metric to evaluate how well our Knowledge graph is constructed. It would require lots of ground truth data.. We propose an already made implementation: [Pykeen](https://pykeen.readthedocs.io/en/stable/tutorial/understanding_evaluation.html)

Loading the necessary libraries

In [ ]:
%%capture
!pip install pykeen

In [ ]:
import pykeen
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline
import pandas as pd
import numpy as np

INFO:pykeen.utils:Using opt_einsum


In [ ]:
import requests

pdf_url = 'https://josephbeasse.fr/fake_triples.csv'

response = requests.get(pdf_url)

if response.status_code == 200:
    with open('fake_triples.csv', 'wb') as file:
        file.write(response.content)
        print("CSV Downloaded.")

else:
    print("Failed to download the CSV file.")

CSV Downloaded.


In [ ]:
csv_path = 'fake_triples.csv'

df = pd.read_csv(csv_path)
triples_array = df.to_numpy()
triples = TriplesFactory.from_labeled_triples(triples_array)

In [ ]:
results = pipeline(
    training=triples,
    testing=triples,
    model='TransE',
)

INFO:pykeen.pipeline.api:Using device: None


Training epochs on cuda:0:   0%|          | 0/5 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=5.


Evaluating on cuda:0:   0%|          | 0.00/5.00 [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.08s seconds


In [ ]:
metric_results = results.metric_results
print(metric_results)

In [ ]:
mean_rank = metric_results.get_metric('mean_rank')
hits_at_1 = metric_results.get_metric('hits@1')
hits_at_3 = metric_results.get_metric('hits@3')
hits_at_10 = metric_results.get_metric('hits@10')
mrr = metric_results.get_metric('mean_reciprocal_rank')

print(f"Mean Rank: {mean_rank}")
print(f"Hits@1: {hits_at_1}")
print(f"Hits@3: {hits_at_3}")
print(f"Hits@10: {hits_at_10}")
print(f"MRR: {mrr}")

Mean Rank: 3.5999999046325684
Hits@1: 0.0
Hits@3: 0.5
Hits@10: 1.0
MRR: 0.2983333468437195


In [ ]:
import random
from typing import List, Tuple
import numpy as np
import torch
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline

# Adjusting the dummy data format to be compatible with PyKEEN
def generate_dummy_data_array(num_entities: int, num_relations: int, num_triples: int) -> np.ndarray:
    entities = [f"Entity{i}" for i in range(num_entities)]
    relations = [f"Relation{i}" for i in range(num_relations)]
    triples = np.array([(random.choice(entities), random.choice(relations), random.choice(entities)) for _ in range(num_triples)])
    return triples

# Redefining the dummy data parameters
num_entities = 10
num_relations = 5
num_triples = 50

# Generate dummy data in the correct format
triples_array = generate_dummy_data_array(num_entities, num_relations, num_triples)

# Create a triples factory from the dummy data
triples_factory = TriplesFactory.from_labeled_triples(triples_array)

# Split the dataset into training, validation, and testing
training, validation, testing = triples_factory.split([0.8, 0.1, 0.1])

# Define a PyKEEN pipeline for training a model and evaluating it
result = pipeline(
    training=training,
    testing=testing,
    model='TransE',
    training_loop='slcwa',
    loss='MarginRankingLoss',
    optimizer='Adam',
    optimizer_kwargs=dict(lr=0.01),
    training_kwargs=dict(num_epochs=5, batch_size=32),
    evaluation_kwargs=dict(batch_size=32),
)

# Get the evaluation results
evaluation_results = result.metric_results.to_dict()

evaluation_results


INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [28, 5, 5]
INFO:pykeen.pipeline.api:Using device: None


Training epochs on cuda:0:   0%|          | 0/5 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Evaluating on cuda:0:   0%|          | 0.00/5.00 [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.09s seconds


{'head': {'optimistic': {'inverse_arithmetic_mean_rank': 0.16129032258064516,
   'adjusted_inverse_harmonic_mean_rank': -0.19657146890131796,
   'geometric_mean_rank': 6.0438002711712695,
   'count': 5.0,
   'standard_deviation': 1.32664991614216,
   'adjusted_arithmetic_mean_rank_index': -0.23809523809523792,
   'arithmetic_mean_rank': 6.2,
   'median_absolute_deviation': 1.482602218505602,
   'z_inverse_harmonic_mean_rank': -1.1433341087854805,
   'z_arithmetic_mean_rank': -0.82572282384477,
   'harmonic_mean_rank': 5.874125874125875,
   'inverse_harmonic_mean_rank': 0.17023809523809522,
   'adjusted_arithmetic_mean_rank': 1.1923076923076923,
   'variance': 1.7600000000000002,
   'inverse_geometric_mean_rank': 0.1654588098766214,
   'median_rank': 6.0,
   'inverse_median_rank': 0.16666666666666666,
   'adjusted_geometric_mean_rank_index': -0.4470036097045944,
   'z_geometric_mean_rank': -1.1832025005201428,
   'hits_at_1': 0.0,
   'hits_at_3': 0.0,
   'hits_at_5': 0.2,
   'hits_at_10

**Explanation**

**Mean Rank** (Arithmetic Mean Rank): This indicates the average rank of the true triplets. A lower mean rank is desirable as it indicates that the model ranks true triplets higher.

**Hits@K**: This is the proportion of correct entities found in the top-K predictions. For instance, Hits@1, Hits@3, and Hits@10 show the percentage of true entities that appear in the top 1, 3, and 10 positions of the ranked list, respectively. Higher values are better, indicating that the model often ranks true triples highly.

**Mean Reciprocal Rank** (Inverse Harmonic Mean Rank): This is the average of the reciprocal ranks of the true triplets. Higher values (closer to 1) are better, indicating that the true triplets tend to be ranked highly.

**Geometric Mean Rank and Variance**: These give an idea of the distribution of the ranks. The geometric mean is less sensitive to outliers than the arithmetic mean.

**Standard Deviation**: Shows the variability in the ranks. A high standard deviation indicates that the performance of the model is inconsistent across different queries.

**How to analyze ?**

The mean ranks (around 6 for head and 4 for tail entities) suggest that the true triples are not consistently ranked at the very top.

Hits@1 is 0 for both head and tail, meaning the true entity is rarely the top prediction.

However, Hits@10 is 1.0, indicating that the model ranks the true entity in the top 10 predictions consistently.

The standard deviation is relatively high, suggesting variability in the model's performance.